In [61]:
import numpy as np
import pandas as pd
from haversine import haversine
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

# 데이터 전처리

In [5]:
airline = pd.read_csv("airline_data/flightlist_20190101_20190131.csv")
airport = pd.read_csv("airline_data/airport.csv")

/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
airline

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,HVN19,NaN,888152,NaN,NaN,YMML,LFPG,2018-12-31 00:43:16+00:00,2019-01-01 04:56:29+00:00,2019-01-01 00:00:00+00:00,-37.659485,144.804421,304.8,48.995316,2.610802,-53.34
1,CCA839,NaN,780ad1,NaN,NaN,YMML,LEBL,2018-12-31 00:53:08+00:00,2019-01-01 06:01:42+00:00,2019-01-01 00:00:00+00:00,-37.692123,144.841997,304.8,41.036124,2.063557,1112.52
2,CES219,NaN,780b7e,B-5936,A332,YSSY,EDDF,2018-12-31 01:05:29+00:00,2019-01-01 04:09:29+00:00,2019-01-01 00:00:00+00:00,-33.930908,151.171987,0.0,50.045563,8.588923,-15.24
3,AEA040,NaN,34444e,EC-LVL,A332,LEMD,LEMD,2018-12-31 01:07:21+00:00,2019-01-01 03:32:59+00:00,2019-01-01 00:00:00+00:00,40.534756,-3.575426,609.6,40.475728,-3.538347,411.48
4,CXA825,NaN,780d75,B-2760,B788,YSSY,LFPG,2018-12-31 01:18:29+00:00,2019-01-01 04:32:28+00:00,2019-01-01 00:00:00+00:00,-33.954254,151.178041,0.0,48.996091,2.625805,22.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066564,QTR8294,NaN,06a192,A7-AFZ,A332,LSZM,OTHH,2019-01-30 20:32:22+00:00,2019-01-31 02:26:42+00:00,2019-01-31 00:00:00+00:00,47.594636,7.528857,304.8,25.212387,51.645762,220.98
2066565,ENY3774,NaN,a252a7,NaN,NaN,KDFW,KMSN,2019-01-30 20:32:25+00:00,2019-01-30 22:29:35+00:00,2019-01-30 00:00:00+00:00,32.895859,-97.030042,0.0,43.152878,-89.340820,220.98
2066566,THY29X,NaN,4baa55,TC-JRU,A321,LTBA,DAAG,2019-01-30 20:32:25+00:00,2019-01-30 23:51:24+00:00,2019-01-30 00:00:00+00:00,41.029048,28.806652,609.6,36.689029,3.297418,472.44
2066567,N178FA,NaN,a13763,N178FA,NaN,PS87,4MD8,2019-01-30 20:32:26+00:00,2019-01-30 20:59:48+00:00,2019-01-30 00:00:00+00:00,40.029465,-80.151917,2133.6,39.376556,-77.338735,510.54


In [48]:
for idx, coordinate in enumerate(tqdm(airport["coordinates"])): 
    temp = coordinate.split(", ")
    airport.loc[idx,"latitude"] = temp[1]
    airport.loc[idx,"longitude"] = temp[0]
    
airport["latitude"] = airport["latitude"].astype(float)
airport["longitude"] = airport["longitude"].astype(float)

In [49]:
airport

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125",40.070801,-74.933601
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022",38.704022,-101.473911
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968",59.949200,-151.695999
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172",34.864799,-86.770302
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087",35.608700,-91.254898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57348,ZYYK,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524",40.542524,122.358600
57349,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406",41.784401,123.496002
57350,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444",51.894444,1.482500
57351,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999",-11.584278,47.296389


In [ ]:
tor = 6
print(len(airline[airline["origin"].isnull()]))

for idx, column in tqdm(airline[airline["origin"].isnull()].iterrows()):
    airport_coordinate = airport.loc[(airport["latitude"] > column["latitude_1"] - tor) & (airport["latitude"] < column["latitude_1"] + tor) & (airport["longitude"] > column["longitude_1"] - tor) & (airport["longitude"] < column["longitude_1"] + tor)]
    origin_coordinate = (column["latitude_1"],column["longitude_1"])
    dist = {colums["ident"] : haversine(origin_coordinate, (colums["latitude"], colums["longitude"])) for idx, colums in airport_coordinate.iterrows()}
    column["origin"] = sorted(dist.items(), key = lambda x : x[1])[0][0]
    
airline[airline["origin"].isnull()]

630968


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed